In [1]:
import os
import joblib
import numpy as np
import pandas as pd
from glob import glob
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Parámetros del proyecto
PROJECT_ROOT    = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATA_INTERIM    = os.path.join(PROJECT_ROOT, 'data', 'interim')

TRAIN_CVS_PATH  = os.path.join(DATA_INTERIM, 'cvs_texto_train.csv')
TEST_CVS_PATH   = os.path.join(DATA_INTERIM, 'cvs_texto_test.csv')

OFFER_FILES = {
    'Ciencia_de_datos':    os.path.join(DATA_INTERIM, 'Ciencia_de_datos_España_ofertas.csv'),
    'Ingeniero_de_datos':  os.path.join(DATA_INTERIM, 'Ingeniero_de_datos_España_ofertas.csv'),
    'Jurista':             os.path.join(DATA_INTERIM, 'Jurista_Málaga_ofertas.csv'),
    'Traductor_de_inglés': os.path.join(DATA_INTERIM, 'Traductor_de_inglés_Málaga_ofertas.csv')
}

MAPPING = {
    'cdatos':    'Ciencia_de_datos',
    'ingdatos':  'Ingeniero_de_datos',
    'jurista':   'Jurista',
    'traductor': 'Traductor_de_inglés'
}

RNG_SEED = 42


In [10]:
SPANISH_STOPWORDS = stopwords.words('spanish')

In [3]:
def load_cvs(path, mapping):
    df = pd.read_csv(path)
    df['code'] = df['Nombre del archivo'].str.split('_').str[2]
    df['category'] = df['code'].map(mapping)
    return df.rename(columns={'Texto extraído':'cv_text','Nombre del archivo':'cv_id'})[['cv_id','cv_text','category']]

def load_offers(offer_files):
    offers = []
    for cat, fp in offer_files.items():
        df = pd.read_csv(fp).rename(columns={'descripcion_oferta':'offer_text'})
        df['offer_id'] = df.index.astype(str) + f'_{cat}'
        df['offer_category'] = cat
        offers.append(df[['offer_id','offer_text','offer_category']])
    return pd.concat(offers, ignore_index=True)


In [21]:
from sklearn.svm import SVC

pipeline_proba = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    # Cambiamos LinearSVC por SVC con probability=True
    ('clf',   SVC(
                  kernel='linear', 
                  probability=True, 
                  class_weight='balanced',
                  random_state=RNG_SEED
              ))
])


In [5]:
# ─── 2. Carga de los CVs de entrenamiento ────────────────────────────────
df_cvs = load_cvs(TRAIN_CVS_PATH, MAPPING)
# Ahora df_cvs tiene columnas: ['cv_id', 'cv_text', 'category']

In [8]:
# ─── 3. Carga de las ofertas y etiquetado ─────────────────────────────────
offers = load_offers(OFFER_FILES)
# Ahora offers tiene columnas: ['offer_id', 'offer_text', 'offer_category']

In [9]:
# ─── 4. Construcción del dataset de pares CV–oferta ───────────────────────
#  — le ponemos una llave auxiliar para hacer producto cartesiano
df_cvs = df_cvs.assign(key=1)
offers = offers.assign(key=1)

pairs = df_cvs.merge(offers, on='key').drop('key', axis=1)
# pairs now has:
#   ['cv_id','cv_text','category','offer_id','offer_text','offer_category']

# Etiqueta binaria: 1 si coincide la categoría, 0 si no
pairs['label'] = (pairs['category'] == pairs['offer_category']).astype(int)

# Renombramos para mayor claridad
pairs = pairs.rename(columns={
    'category': 'cv_category'
})
# (ya tenemos offer_category)

In [22]:
# ─── 5. TF–IDF + SVM con GridSearchCV ───────────────────────────────────

# 5.1 Prepara X e y
X = pairs['cv_text'] + ' ' + pairs['offer_text']
y = pairs['label']

# 5.2 Define la validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RNG_SEED)

param_grid_proba = {
    # TF–IDF
    'tfidf__stop_words':   [SPANISH_STOPWORDS],
    'tfidf__ngram_range':  [(1,1),(1,2)],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__min_df':       [1,2,3],
    'tfidf__max_df':       [0.5,0.75,1.0],
    # SVC
    'clf__C':              [0.01,0.1,1,10],
    'clf__kernel': ['linear','rbf']
}

# 5.4 GridSearchCV
search = GridSearchCV(
    estimator=pipeline_proba,
    param_grid=param_grid_proba,
    scoring='f1',
    cv=cv,
    n_jobs=-1,
    verbose=2,
    refit=True
)

# 5.5 Ejecuta la búsqueda
search.fit(X, y)

# 5.6 Resultados
print(f"▶ Mejor F1 (CV): {search.best_score_:.4f}\n")
print("▶ Mejores parámetros:")
for p, v in search.best_params_.items():
    print(f"  • {p}: {v}")

# 5.7 Top 10 combinaciones
results = (
    pd.DataFrame(search.cv_results_)
      .loc[:, ['params','mean_test_score','std_test_score']]
      .sort_values('mean_test_score', ascending=False)
      .head(10)
)
print("\nTop 10 combinaciones:")
display(results)


Fitting 5 folds for each of 288 candidates, totalling 1440 fits
▶ Mejor F1 (CV): 0.8278

▶ Mejores parámetros:
  • clf__C: 10
  • clf__kernel: rbf
  • tfidf__max_df: 1.0
  • tfidf__min_df: 1
  • tfidf__ngram_range: (1, 1)
  • tfidf__stop_words: ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos

,params,mean_test_score,std_test_score
277,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.827832,0.041375
285,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.827832,0.041375
281,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.827832,0.041375
273,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.818884,0.047921
265,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.818884,0.047921
269,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.818884,0.047921
253,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.810240,0.044179
257,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.810240,0.044179
261,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.810240,0.044179
276,"{'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__m...",0.784371,0.037210


In [23]:
# ─── 6. Guardar el mejor modelo ────────────────────────────────────────
# Busca tu mejor estimador en el GridSearch definido (search o search_refined)
best_model = search.best_estimator_   
print("Parámetros del mejor modelo:", search.best_params_)

# Crea la carpeta para modelos si no existe
MODEL_DIR = os.path.join(PROJECT_ROOT, 'models')
os.makedirs(MODEL_DIR, exist_ok=True)

# Guarda el pipeline completo
joblib.dump(best_model, os.path.join(MODEL_DIR, 'svm_tfidf_best.pkl'))
print("Modelo serializado en:", os.path.join(MODEL_DIR, 'svm_tfidf_best.pkl'))


Parámetros del mejor modelo: {'clf__C': 10, 'clf__kernel': 'rbf', 'tfidf__max_df': 1.0, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1), 'tfidf__stop_words': ['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestr

In [24]:
# ─── 7. Ranking de ofertas con probabilidades reales ─────────────────

df_test = load_cvs(TEST_CVS_PATH, MAPPING)

rankings = []
for _, row in df_test.iterrows():
    cv_id   = row['cv_id']
    text_cv = row['cv_text']
    
    # Monta los textos CV+oferta
    batch = [ text_cv + ' ' + off for off in offers['offer_text'] ]
    # Obtén la probabilidad de "match" (clase 1)
    probs = best_model.predict_proba(batch)[:,1]
    
    df_rank = pd.DataFrame({
        'cv_id'         : cv_id,
        'offer_id'      : offers['offer_id'],
        'offer_category': offers['offer_category'],
        'score'         : probs   # aquí valores ∈ [0,1]
    }).sort_values('score', ascending=False).reset_index(drop=True)
    df_rank['rank'] = df_rank.index + 1
    rankings.append(df_rank)

df_rankings = pd.concat(rankings, ignore_index=True)

OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'reports')
os.makedirs(OUTPUT_DIR, exist_ok=True)
out_path = os.path.join(OUTPUT_DIR, 'rankings_test.csv')
df_rankings.to_csv(out_path, index=False)
print("Rankings de test guardados en:", out_path)


Rankings de test guardados en: c:\Users\Pablo\OneDrive - Universidad Loyola Andalucía\Master\TFM\cv-matching\reports\rankings_test.csv
